In [1]:
from explaneat.core import backprop
from explaneat.core.backproppop import BackpropPopulation

from explaneat.core.neuralneat import NeuralNeat as NeuralNeat

import random

import torch
import torch.nn as nn
import torch.optim as optim

import neat

from explaneat.core.utility import MethodTimer


from explaneat.core.backprop import NeatNet
from explaneat.core import backprop
from explaneat.core.backproppop import BackpropPopulation
# from explaneat.visualization import visualize
from explaneat.core.experiment import ExperimentReporter

from explaneat.core.neuralneat import NeuralNeat as nneat

import torch.optim as optim

from torch.functional import F








In [2]:
RANDOM_SEED      = 42
torch.manual_seed(RANDOM_SEED)


USE_CUDA = torch.cuda.is_available()
USE_CUDA = False
device = torch.device("cuda:1" if USE_CUDA else "cpu")


In [3]:

def xor(a, b, threshold = 0.5):
    response = False
    if a > threshold and b < threshold:
        response = True
    if a < threshold and b > threshold:
        response = True
    # return (1.0, 0.0) if response else (0.0, 1.0)
    return 1.0 if response else 0.0
    

def create_n_points(n, size, min=0.0, max=1.0):
    data = []
    for _ in range(n):
        data.append([
            random.uniform(min, max) for ii in range(size)
        ])

    return data

# correct solution:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference

def overUnder(val, threshold):
    return 1. if val > threshold else 0

xor_inputs = create_n_points(400, 2, -1, 1)

xor_outputs = [
    [xor(tup[0], tup[1], 0)] for tup in xor_inputs
]

In [4]:
config_path = "./config_xor"
base_config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)

def instantiate_population(config, xs, ys, saveLocation):

    if not os.path.exists(saveLocation):
        os.makedirs(saveLocation)
        
    config.save(os.path.join(saveLocation, 'config.conf'))

    # Create the population, which is the top-level object for a NEAT run.
    with MethodTimer("Backprop everything"):
        p = BackpropPopulation(config, 
                                xs, 
                                ys, 
                                criterion=nn.BCEWithLogitsLoss())

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5, filename_prefix=str(saveLocation) + "checkpoint-" ))
    bpReporter = backprop.BackpropReporter(True)
    p.add_reporter(bpReporter)
    p.add_reporter(ExperimentReporter(saveLocation))
    
    return p

def eval_genomes(genomes, config):
    
    print(genomes)
    loss = nn.BCELoss()
    loss = loss.to(device)
    for genome_id, genome in genomes.items():
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        preds = []
        for xi in xor_inputs:
            preds.append(net.activate(xi))
        genome.fitness = float(1./loss(torch.tensor(preds).to(device), torch.tensor(xor_outputs)))

config = base_config
saveLocation = './'
maxNGenerations = 20
p = instantiate_population(config, xor_inputs, xor_outputs, saveLocation)
# Run for up to nGenerations generations.
winner = p.run(eval_genomes, maxNGenerations, nEpochs = 5)

g = p.best_genome


The function - Backprop everything - has just started at 1632349599.609369
The function - Backprop everything - took 0.003626108169555664 seconds to complete
The function - generationStart - has just started at 1632349599.61352

 ****** Running generation 0 ****** 

The function - generationStart - took 5.316734313964844e-05 seconds to complete
The function - pre_backprop - has just started at 1632349599.613655
The function - pre_backprop - took 0.00014209747314453125 seconds to complete
The function - backprop - has just started at 1632349599.613813
about to start backprop with 5 epochs
mean improvement: -0.0012304783675389918
best improvement: -0.0018211194786518226
best loss: 0.4977222902033316
The function - backprop - took 0.2835700511932373 seconds to complete
The function - post_backprop - has just started at 1632349599.897401
The function - post_backprop - took 2.574920654296875e-05 seconds to complete
The function - evaluate fitness - has just started at 1632349599.897501
{1: 

In [5]:
print(g)

Key: 1742
Fitness: 1.6242040395736694
Nodes:
	0 DefaultNodeGene(key=0, bias=0.062185441321651026, response=1.0, activation=sigmoid, aggregation=sum)
	17 DefaultNodeGene(key=17, bias=-1.9928506351210846, response=1.0, activation=sigmoid, aggregation=sum)
	94 DefaultNodeGene(key=94, bias=-0.007069086670578928, response=1.0, activation=sigmoid, aggregation=sum)
	131 DefaultNodeGene(key=131, bias=1.4244605850425758, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-2, 0), weight=-0.24685938532872398, enabled=False)
	DefaultConnectionGene(key=(-2, 17), weight=-0.6023529937709702, enabled=False)
	DefaultConnectionGene(key=(-2, 94), weight=1.2808861888543372, enabled=True)
	DefaultConnectionGene(key=(-2, 131), weight=-1.6531205284482435, enabled=True)
	DefaultConnectionGene(key=(-1, 0), weight=0.485071600289533, enabled=False)
	DefaultConnectionGene(key=(-1, 17), weight=-0.46461544903263535, enabled=True)
	DefaultConnectionGene(key=(17, 0), weight=-1

In [6]:
net = nneat(g, p.config, criterion=nn.BCEWithLogitsLoss())


In [7]:
net.genome.connections
# Error is in inputs not being updated with extra info from skip layers

{(-1, 0): <neat.genes.DefaultConnectionGene at 0x7fc92145d610>,
 (-2, 0): <neat.genes.DefaultConnectionGene at 0x7fc92145d670>,
 (-2, 17): <neat.genes.DefaultConnectionGene at 0x7fc92145d6d0>,
 (17, 0): <neat.genes.DefaultConnectionGene at 0x7fc92145d730>,
 (-1, 17): <neat.genes.DefaultConnectionGene at 0x7fc92145d790>,
 (-2, 94): <neat.genes.DefaultConnectionGene at 0x7fc92145d7f0>,
 (94, 17): <neat.genes.DefaultConnectionGene at 0x7fc92145d850>,
 (-2, 131): <neat.genes.DefaultConnectionGene at 0x7fc92145d8b0>,
 (131, 17): <neat.genes.DefaultConnectionGene at 0x7fc92145d910>}

In [8]:
net.forward(torch.tensor( xor_inputs[:10], dtype=torch.float64) ) [:10]

tensor([[0.3512],
        [0.4069],
        [0.3984],
        [0.3909],
        [0.4065],
        [0.3435],
        [0.3893],
        [0.3909],
        [0.3828],
        [0.3580]], dtype=torch.float64, grad_fn=<SliceBackward>)

In [9]:
net.forward(torch.tensor( xor_inputs, dtype=torch.float64) )[:20]

tensor([[0.3512],
        [0.4069],
        [0.3984],
        [0.3909],
        [0.4065],
        [0.3435],
        [0.3893],
        [0.3909],
        [0.3828],
        [0.3580],
        [0.3904],
        [0.4095],
        [0.4214],
        [0.3692],
        [0.4133],
        [0.4092],
        [0.4505],
        [0.4114],
        [0.4486],
        [0.3621]], dtype=torch.float64, grad_fn=<SliceBackward>)

In [10]:
# net.optimiser.zero_grad()

In [11]:
# net.optimise(xor_inputs, xor_outputs)

In [12]:
optimiser = optim.Adadelta(net.parameters())

In [13]:
net.weights

{3: Parameter containing:
 tensor([[ 0.0000],
         [ 0.0000],
         [-1.5946]], dtype=torch.float64, requires_grad=True),
 2: Parameter containing:
 tensor([[-0.4646],
         [ 0.0000],
         [-0.4330],
         [ 1.6059]], dtype=torch.float64, requires_grad=True),
 1: Parameter containing:
 tensor([[ 0.0000,  0.0000],
         [ 1.2809, -1.6531]], dtype=torch.float64, requires_grad=True),
 0: Parameter containing:
 tensor([], size=(0, 2), dtype=torch.float64, requires_grad=True)}

In [14]:
optimizer = optimiser
losses = []
for i in range(10000):
    preds = net.forward(torch.tensor( xor_inputs, dtype=torch.float64) )
    loss = F.mse_loss(preds, torch.tensor( xor_outputs, dtype=torch.float64)).sqrt()
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    losses.append(loss)  
losses[-10:]


[tensor(0.1602, dtype=torch.float64, grad_fn=<SqrtBackward>),
 tensor(0.1601, dtype=torch.float64, grad_fn=<SqrtBackward>),
 tensor(0.1602, dtype=torch.float64, grad_fn=<SqrtBackward>),
 tensor(0.1601, dtype=torch.float64, grad_fn=<SqrtBackward>),
 tensor(0.1602, dtype=torch.float64, grad_fn=<SqrtBackward>),
 tensor(0.1601, dtype=torch.float64, grad_fn=<SqrtBackward>),
 tensor(0.1602, dtype=torch.float64, grad_fn=<SqrtBackward>),
 tensor(0.1601, dtype=torch.float64, grad_fn=<SqrtBackward>),
 tensor(0.1602, dtype=torch.float64, grad_fn=<SqrtBackward>),
 tensor(0.1601, dtype=torch.float64, grad_fn=<SqrtBackward>)]

In [15]:
preds[:10]

tensor([[2.3804e-03],
        [9.9987e-01],
        [8.2855e-01],
        [9.7352e-01],
        [9.9511e-01],
        [5.7201e-03],
        [9.9999e-01],
        [9.9772e-01],
        [5.1593e-01],
        [2.2643e-04]], dtype=torch.float64, grad_fn=<SliceBackward>)

In [16]:
net.weights

{3: Parameter containing:
 tensor([[ -3.1107],
         [ -6.1680],
         [-21.4574]], dtype=torch.float64, requires_grad=True),
 2: Parameter containing:
 tensor([[-1.6426],
         [ 8.7047],
         [-9.5125],
         [12.7464]], dtype=torch.float64, requires_grad=True),
 1: Parameter containing:
 tensor([[ 5.1164,  3.9662],
         [ 4.9328, -4.0456]], dtype=torch.float64, requires_grad=True),
 0: Parameter containing:
 tensor([], size=(0, 2), dtype=torch.float64, requires_grad=True)}

In [17]:
net.layers

{3: {'nodes': {0: {'depth': 3,
    'output_ids': [],
    'input_ids': [-1, -2, 17],
    'output_layers': [],
    'needs_skip': False,
    'id': 0,
    'input_layers': [0, 0, 2],
    'skip_layer_input': True,
    'layer_index': 0}},
  'is_output_layer': True,
  'is_input_layer': False,
  'layer_type': 'OUTPUT',
  'input_layers': [0, 2],
  'input_shape': 3,
  'weights_shape': (3, 1),
  'out_weights': [],
  'bias': [0.062185441321651026],
  'in_weights': [[0]],
  'input_weights': array([[ 0.        ],
         [ 0.        ],
         [-1.59462846]]),
  'input_map': {(17, 0): (2, 0)}},
 2: {'nodes': {17: {'depth': 2,
    'output_ids': [0],
    'input_ids': [-2, -1, 94, 131],
    'output_layers': [3],
    'needs_skip': False,
    'id': 17,
    'input_layers': [0, 0, 1, 1],
    'skip_layer_input': True,
    'layer_index': 0}},
  'is_output_layer': False,
  'is_input_layer': False,
  'layer_type': 'CONNECTED',
  'input_layers': [0, 1],
  'input_shape': 4,
  'weights_shape': (4, 1),
  'out_wei

In [18]:
net.node_tracker[0]['depth']

3

In [19]:


reached_nodes = []

node_stack = []

# Instantiate stack with depth==0 nodes
for node_id, node in net.node_tracker.items():
    if node_id < 0:
        node_stack.append(node_id)

while len(node_stack) > 0:
    reached_nodes.append(node_stack[0])
    for node_id in net.node_tracker[node_stack[0]]['output_ids']:
        node_stack.append(node_id)
    del(node_stack[0])

if True:
    print("the reached nodes are")
    print(reached_nodes)


for node_id in net.node_tracker:
    if not node_id in reached_nodes:
        print(reached_nodes)
        print(net.node_tracker)
        print("I can't reach this node {}".format(node_id))

the reached nodes are
[-1, -2, 0, 17, 0, 17, 94, 131, 0, 0, 17, 17, 0, 0]


In [20]:
print(net.genome)

Key: 1742
Fitness: 1.6242040395736694
Nodes:
	0 DefaultNodeGene(key=0, bias=0.062185441321651026, response=1.0, activation=sigmoid, aggregation=sum)
	17 DefaultNodeGene(key=17, bias=-1.9928506351210846, response=1.0, activation=sigmoid, aggregation=sum)
	94 DefaultNodeGene(key=94, bias=-0.007069086670578928, response=1.0, activation=sigmoid, aggregation=sum)
	131 DefaultNodeGene(key=131, bias=1.4244605850425758, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-2, 0), weight=-0.24685938532872398, enabled=False)
	DefaultConnectionGene(key=(-2, 17), weight=-0.6023529937709702, enabled=False)
	DefaultConnectionGene(key=(-2, 94), weight=1.2808861888543372, enabled=True)
	DefaultConnectionGene(key=(-2, 131), weight=-1.6531205284482435, enabled=True)
	DefaultConnectionGene(key=(-1, 0), weight=0.485071600289533, enabled=False)
	DefaultConnectionGene(key=(-1, 17), weight=-0.46461544903263535, enabled=True)
	DefaultConnectionGene(key=(17, 0), weight=-1